In [1]:
import pandas as pd
import json

import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Add src directory to path for k_base module
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)


from analysis.compute_sigtests import calculate_match_with_filters

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
mami = pd.read_csv("../data/collab-rec-2026/analysis/gemini_moderator_success_aggressive_mami.csv")
masi = pd.read_csv("../data/collab-rec-2026/analysis/gemini_moderator_success_aggressive_masi.csv")

In [4]:
mami.rename(columns={"mod_succ_relevance_score": "mami_success_score",\
                     "round_nr": "mami_round",\
                     "final_offer": "final_offer_mami"}, inplace=True)
mami.head()

,exp_config,query_id,final_offer_mami,mami_round,mami_success_score
0,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_30_pop_high_hard,"['Copenhagen', 'Amsterdam', 'Prague', 'Stockho...",10,0.725
1,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_50_pop_low_hard,"['Chisinau', 'Kaunas', 'Aalborg', 'Bari', 'Bru...",10,0.475
2,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_155_pop_low_hard,"['Ljubljana', 'Chisinau', 'Bratislava', 'Cluj-...",10,0.525
3,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_95_pop_medium_hard,"['Bologna', 'Porto', 'Lyon', 'Valencia', 'Thes...",10,0.675
4,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_15_pop_medium_hard,"['Bologna', 'Berlin', 'Budapest', 'Prague', 'K...",10,0.475


In [5]:
masi.rename(columns={"mod_succ_relevance_score": "masi_success_score", \
                     "final_offer": "final_offer_masi",\
                     "round_nr": "masi_round"
                    }, inplace=True)
masi.head()

,exp_config,query_id,final_offer_masi,masi_round,masi_success_score
0,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_30_pop_high_hard,"['Paris', 'Copenhagen', 'Innsbruck', 'Ljubljan...",1,0.675
1,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_50_pop_low_hard,"['Budapest', 'Prague', 'Chisinau', 'Krakow', '...",1,0.525
2,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_155_pop_low_hard,"['Belgrade', 'Ljubljana', 'Chisinau', 'Krakow'...",1,0.525
3,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_95_pop_medium_hard,"['Bologna', 'Copenhagen', 'Paris', 'Porto', 'L...",1,0.725
4,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_15_pop_medium_hard,"['Bologna', 'Berlin', 'Copenhagen', 'Amsterdam...",1,0.550


In [10]:
def find_level_pop(config_id):
    pop_level = config_id.split("_")[3:]
    return "_".join(pop_level)

In [12]:
df = masi.merge(mami, on=["exp_config", "query_id"], how="right")
df.drop(columns=["mami_round", "masi_round"], inplace=True)
df["pop_level"] = df["query_id"].apply(find_level_pop)
df

,exp_config,query_id,final_offer_masi,masi_success_score,final_offer_mami,mami_success_score,pop_level
0,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_30_pop_high_hard,"['Paris', 'Copenhagen', 'Innsbruck', 'Ljubljan...",0.675000,"['Copenhagen', 'Amsterdam', 'Prague', 'Stockho...",0.725000,pop_high_hard
1,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_50_pop_low_hard,"['Budapest', 'Prague', 'Chisinau', 'Krakow', '...",0.525000,"['Chisinau', 'Kaunas', 'Aalborg', 'Bari', 'Bru...",0.475000,pop_low_hard
2,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_155_pop_low_hard,"['Belgrade', 'Ljubljana', 'Chisinau', 'Krakow'...",0.525000,"['Ljubljana', 'Chisinau', 'Bratislava', 'Cluj-...",0.525000,pop_low_hard
3,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_95_pop_medium_hard,"['Bologna', 'Copenhagen', 'Paris', 'Porto', 'L...",0.725000,"['Bologna', 'Porto', 'Lyon', 'Valencia', 'Thes...",0.675000,pop_medium_hard
4,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_15_pop_medium_hard,"['Bologna', 'Berlin', 'Copenhagen', 'Amsterdam...",0.550000,"['Bologna', 'Berlin', 'Budapest', 'Prague', 'K...",0.475000,pop_medium_hard
5,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_93_pop_high_hard,"['Krakow', 'Paris', 'Prague', 'Budapest', 'Rom...",0.750000,"['Prague', 'Budapest', 'Berlin', 'Amsterdam', ...",0.750000,pop_high_hard
6,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_104_pop_high_hard,"['Krakow', 'Ljubljana', 'London', 'Budapest', ...",0.625000,"['Krakow', 'Budapest', 'Prague', 'Porto', 'Ank...",0.550000,pop_high_hard
7,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_173_pop_high_hard,"['Paris', 'Rome', 'Ljubljana', 'Berlin', 'Barc...",0.775000,"['Berlin', 'Barcelona', 'Budapest', 'Prague', ...",0.650000,pop_high_hard
8,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_12_pop_high_hard,"['Rome', 'Ljubljana', 'Bologna', 'Krakow', 'Pa...",0.750000,"['Vienna', 'Ancona', 'Bari', 'Brussels', 'Cagl...",0.450000,pop_high_hard
9,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_187_pop_high_hard,"['Prague', 'Copenhagen', 'London', 'Berlin', '...",0.625000,"['Prague', 'Berlin', 'Amsterdam', 'Vienna', 'B...",0.600000,pop_high_hard


In [ ]:
df["config_pop_level"].apply

In [21]:
config_id = "c_p_27_pop_high_hard"
query= "\"Recommend European cities with high popularity and budget that offer a variety of cultural events, nightlife, and historical landmarks to visit in May.\""

filters = {
        "popularity": "high",
        "interests": "Arts & Entertainment",
        "month": "May",
        "budget": "high"
      }
pop_r1_candidates = [
          "Prague",
          "Budapest",
          "Krakow",
          "Berlin",
          "Barcelona",
          "Madrid",
          "Rome",
          "Amsterdam",
          "London",
          "Paris"
        ]
per_r1_candidates =  [
          "Budapest",
          "Prague",
          "Belgrade",
          "Warsaw",
          "Krakow",
          "Sofia",
          "Porto",
          "Valencia",
          "Thessaloniki",
          "Berlin"
        ]
sus_r1_candidates = [
          "Ljubljana",
          "Vilnius",
          "Riga",
          "Tallinn",
          "Krakow",
          "Warsaw",
          "Budapest",
          "Berlin",
          "Porto",
          "Belgrade"
        ]
masi_candidates = [
          "Paris",
          "London",
          "Berlin",
          "Amsterdam",
          "Vienna",
          "Rome",
          "Barcelona",
          "Prague",
          "Madrid",
          "Copenhagen"
        ]
mami_candidates = [
          "Berlin",
          "Amsterdam",
          "Barcelona",
          "Prague",
          "Budapest",
          "Milan",
          "Istanbul",
          "Batman",
          "Belgorod",
          "Belgrade"
        ]

In [20]:
set(masi_candidates).intersection(set(sus_candidates))

{'Berlin'}

In [9]:
calculate_match_with_filters(mami_candidates, filters)


 
city: Berlin matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May', 'budget': 'high'}, unmatched: [], rel_score: 1.0

 
city: Amsterdam matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May', 'budget': 'high'}, unmatched: [], rel_score: 1.0

 
city: Barcelona matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May', 'budget': 'high'}, unmatched: [], rel_score: 1.0

 
city: Prague matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May', 'budget': 'high'}, unmatched: [], rel_score: 1.0

 
city: Budapest matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May'}, unmatched: ['budget'], rel_score: 0.75

 
city: Milan matched  filters: {'popularity': 'high', 'interests': 'Arts & Entertainment', 'month': 'May', 'budget': 'high'}, unmatched: [], rel_score: 1.0

 
city: Istanbul matched  filters: {'popularity

0.75

In [ ]:
# had round_nr from 21 "c_p_123_pop_low_hard"

In [10]:
masi_pop_offer = ['Paris', 'Rome', 'London', 'Berlin', 'Madrid', 'Barcelona', 'Amsterdam', 'Prague', 'Vienna', 'Budapest']
masi_per_offer = 
print(masi_pop_offer)

['Paris', 'Rome', 'London', 'Berlin', 'Madrid', 'Barcelona', 'Amsterdam', 'Prague', 'Vienna', 'Budapest']


In [ ]:
"c_p_79_pop_high_hard","c_p_177_pop_high_medium", "c_p_120_pop_high_medium"

In [14]:
df_pop_low  = df[df["pop_level"].str.contains("pop_low", na=False)]
df_pop_med  = df[df["pop_level"].str.contains("pop_medium", na=False)]
df_pop_high = df[df["pop_level"].str.contains("pop_high", na=False)]


df_pop_low

,exp_config,query_id,final_offer_masi,masi_success_score,final_offer_mami,mami_success_score,pop_level
1,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_50_pop_low_hard,"['Budapest', 'Prague', 'Chisinau', 'Krakow', '...",0.525,"['Chisinau', 'Kaunas', 'Aalborg', 'Bari', 'Bru...",0.475,pop_low_hard
2,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_155_pop_low_hard,"['Belgrade', 'Ljubljana', 'Chisinau', 'Krakow'...",0.525,"['Ljubljana', 'Chisinau', 'Bratislava', 'Cluj-...",0.525,pop_low_hard


In [15]:
df_pop_high

,exp_config,query_id,final_offer_masi,masi_success_score,final_offer_mami,mami_success_score,pop_level
0,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_30_pop_high_hard,"['Paris', 'Copenhagen', 'Innsbruck', 'Ljubljan...",0.675000,"['Copenhagen', 'Amsterdam', 'Prague', 'Stockho...",0.725000,pop_high_hard
5,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_93_pop_high_hard,"['Krakow', 'Paris', 'Prague', 'Budapest', 'Rom...",0.750000,"['Prague', 'Budapest', 'Berlin', 'Amsterdam', ...",0.750000,pop_high_hard
6,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_104_pop_high_hard,"['Krakow', 'Ljubljana', 'London', 'Budapest', ...",0.625000,"['Krakow', 'Budapest', 'Prague', 'Porto', 'Ank...",0.550000,pop_high_hard
7,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_173_pop_high_hard,"['Paris', 'Rome', 'Ljubljana', 'Berlin', 'Barc...",0.775000,"['Berlin', 'Barcelona', 'Budapest', 'Prague', ...",0.650000,pop_high_hard
8,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_12_pop_high_hard,"['Rome', 'Ljubljana', 'Bologna', 'Krakow', 'Pa...",0.750000,"['Vienna', 'Ancona', 'Bari', 'Brussels', 'Cagl...",0.450000,pop_high_hard
9,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_187_pop_high_hard,"['Prague', 'Copenhagen', 'London', 'Berlin', '...",0.625000,"['Prague', 'Berlin', 'Amsterdam', 'Vienna', 'B...",0.600000,pop_high_hard
10,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_27_pop_high_hard,"['Paris', 'London', 'Berlin', 'Amsterdam', 'Vi...",1.000000,"['Berlin', 'Amsterdam', 'Barcelona', 'Prague',...",0.750000,pop_high_hard
11,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_61_pop_high_medium,"['Berlin', 'Copenhagen', 'Paris', 'Amsterdam',...",0.900000,"['Berlin', 'Copenhagen', 'Paris', 'Amsterdam',...",0.866667,pop_high_medium
12,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_126_pop_high_medium,"['Prague', 'Bologna', 'Paris', 'Budapest', 'Po...",0.600000,"['Prague', 'Budapest', 'Krakow', 'Berlin', 'Vi...",0.633333,pop_high_medium
13,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_144_pop_high_medium,"['Berlin', 'London', 'Budapest', 'Amsterdam', ...",0.900000,"['Berlin', 'Budapest', 'Prague', 'Barcelona', ...",0.833333,pop_high_medium


In [17]:
df_pop_med

,exp_config,query_id,final_offer_masi,masi_success_score,final_offer_mami,mami_success_score,pop_level
3,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_95_pop_medium_hard,"['Bologna', 'Copenhagen', 'Paris', 'Porto', 'L...",0.725,"['Bologna', 'Porto', 'Lyon', 'Valencia', 'Thes...",0.675,pop_medium_hard
4,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_15_pop_medium_hard,"['Bologna', 'Berlin', 'Copenhagen', 'Amsterdam...",0.550,"['Bologna', 'Berlin', 'Budapest', 'Prague', 'K...",0.475,pop_medium_hard


In [18]:
df.loc[df["mami_success_score"] > df["masi_success_score"]]

,exp_config,query_id,final_offer_masi,masi_success_score,final_offer_mami,mami_success_score,pop_level
0,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_30_pop_high_hard,"['Paris', 'Copenhagen', 'Innsbruck', 'Ljubljan...",0.675,"['Copenhagen', 'Amsterdam', 'Prague', 'Stockho...",0.725000,pop_high_hard
12,"{'model': 'gemini-2.5-flash', 'rejection_strat...",c_p_126_pop_high_medium,"['Prague', 'Bologna', 'Paris', 'Budapest', 'Po...",0.600,"['Prague', 'Budapest', 'Krakow', 'Berlin', 'Vi...",0.633333,pop_high_medium
